In [1]:
import tensorflow as tf
import pandas as pd
import keras
train = pd.read_csv("dataset_train_set.csv")
test = pd.read_csv("dataset_dev_set.csv")

Using TensorFlow backend.


In [2]:
X_train = train.drop('target',axis=1)
y_train = train['target']
X_test = test.drop('target',axis=1)
y_test = test['target']

In [3]:
X_train = X_train[['aPosLS','hPosLS','hGD5','aSpent']].copy()
X_test = X_test[['aPosLS','hPosLS','hGD5','aSpent']].copy()

In [4]:
X_train.head()

,aPosLS,hPosLS,hGD5,aSpent
0,18,18,0.0,18.07
1,9,5,0.0,8.92
2,12,14,0.0,29.52
3,15,16,0.0,0.00
4,13,3,0.0,31.03


In [5]:
print(X_train.shape)
X_train_norm = tf.keras.utils.normalize(X_train.values,axis=1)
X_test_norm = tf.keras.utils.normalize(X_test.values,axis=1)
X_train_norm[1000]

(4180, 4)


array([ 0.56817265,  0.68180718, -0.02272691,  0.46021984])

In [6]:
y_train.value_counts()

2    1971
0    1120
1    1089
Name: target, dtype: int64

In [7]:
whome = 1
wdraw = 1971/1089
waway = 1971/1120
print("{},{},{}".format(whome,wdraw,waway))

1,1.8099173553719008,1.7598214285714286


In [8]:
class_weights = {0:1.7598,
                 1:1.8099,
                 2:1}

In [9]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
#model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(3, activation = tf.nn.softmax))

In [10]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [11]:
model.fit(X_train_norm, y_train.values, epochs=50, validation_data=(X_test_norm,y_test), class_weight=class_weights)

Train on 4180 samples, validate on 1520 samples
Epoch 1/50
4180/4180 [==============================] - 1s 205us/step - loss: 1.1392 - acc: 0.2691 - val_loss: 1.1015 - val_acc: 0.3349
Epoch 2/50
4180/4180 [==============================] - 0s 53us/step - loss: 1.1057 - acc: 0.3019 - val_loss: 1.0921 - val_acc: 0.4395
Epoch 3/50
4180/4180 [==============================] - 0s 49us/step - loss: 1.0896 - acc: 0.4591 - val_loss: 1.0746 - val_acc: 0.5105
Epoch 4/50
4180/4180 [==============================] - 0s 66us/step - loss: 1.0743 - acc: 0.4888 - val_loss: 1.0677 - val_acc: 0.4974
Epoch 5/50
4180/4180 [==============================] - 0s 44us/step - loss: 1.0652 - acc: 0.4722 - val_loss: 1.0568 - val_acc: 0.4967
Epoch 6/50
4180/4180 [==============================] - 0s 39us/step - loss: 1.0600 - acc: 0.4756 - val_loss: 1.0518 - val_acc: 0.4901
Epoch 7/50
4180/4180 [==============================] - 0s 42us/step - loss: 1.0563 - acc: 0.4768 - val_loss: 1.0473 - val_acc: 0.4875
Epoch 

In [12]:
val_loss, val_acc = model.evaluate(X_test_norm, y_test)
print(val_loss, val_acc)

1520/1520 [==============================] - 0s 35us/step
1.0211318091342323 0.4875


In [13]:
predictions = model.predict_classes(X_test_norm)
preds = list(predictions)
pd.Series(list(predictions)).value_counts()

0    616
2    538
1    366
dtype: int64

In [14]:
predictions[0]

1

In [15]:
correct_pred_dict = {0:0,1:0,2:0}
for i in range(len(preds)):
    pred = preds[i]
    real = y_test[i]
    if pred == real:
        correct_pred_dict[pred] += 1
correct_pred_dict

{0: 276, 1: 113, 2: 352}

In [16]:
outcomes_incorrectly_caught_dict = {0:0,1:0,2:0}
for i in range(len(preds)):
    pred = preds[i]
    real = y_test[i]
    if pred != real:
        outcomes_incorrectly_caught_dict[real] += 1
outcomes_incorrectly_caught_dict

{0: 184, 1: 259, 2: 336}